# Project: Long Beach Animal Shelter Intakes and Outcomes

## Description

**Objective**: \
\
Answer questions for various shareholders in the city of Long Beach, CA concerning intakes and outcomes at the local animal shelter.

**Dataset**: \
\
This dataset was pulled from the [Long Beach Open Data Portal](https://data.longbeach.gov/explore/dataset/animal-shelter-intakes-and-outcomes/). \
It is a 7.8MB CSV file containing intake and outcome data for animals captured by or surrendered to the city.

**Tools Used**:

- pandas
- Matplotlib
- Seaborn

# Introduction

For any city that has at least one animal shelter, there are various shareholders interested in how that shelter is run and what happens to the animals that pass through the shelter's doors.\
\
This analysis looks to answer questions for the following parties in Long Beach, CA:
- **Shelter managers:**
  - How long do animals typically stay in the shelter by species or intake condition?
  - What intake reasons are most strongly correlated with negative outcomes (e.g., euthanasia)?
  - Are there seasonal trends in animal intakes or outcomes?
- **Animal welfare advocates:**
  - What percentage of animals are adopted vs. euthanized, and how does that vary by type, sex, or condition?
  - Are there disparities in outcomes for specific breeds or geographic areas?
  - How many animals are returned to owners vs. adopted?
- **Local government officials:**
  - Is there a correlation between specific neighborhoods and high intake rates?
  - Has the shelter’s performance improved over time (e.g., reduced euthanasia rates)?
  - What’s the annual intake/output volume and trend?
- **Local citizenry:**
  - When is the best time of year to adopt (e.g., more animals available)?
  - What types of animals are most commonly available for adoption?
  - Can geographic patterns inform community outreach for fostering or adoption?
- **Internal analysts:**
  - What features best predict positive outcomes using logistic regression or clustering?
  - Can intake condition be used to forecast outcome types?

# Data handling

## Preview

In [255]:
# Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

# Import helper functions and variables
from utilities.config import get_path_obj, raw_data_path, processed_data_path, products_dir, images_dir, data_dir

In [256]:
# Load data
df = pd.read_csv(raw_data_path, parse_dates=['DOB', 'Intake Date', 'Outcome Date'],)

# Preview
df.head()

,Animal ID,Animal Name,Animal Type,Primary Color,Secondary Color,Sex,DOB,Intake Date,Intake Condition,Intake Type,...,Outcome Type,Outcome Subtype,latitude,longitude,intake_is_dead,outcome_is_dead,was_outcome_alive,geopoint,intake_duration,is_current_month
0,A594350,*HEAVY CREAM,CAT,BLACK,NaN,Neutered,2014-07-28,2017-07-28,NORMAL,STRAY,...,ADOPTION,REPEAT ADT,33.7997598,-118.1263884,Alive on Intake,False,1,"33.7997598, -118.1263884",81.0,0
1,A347815,DUKE,DOG,BLACK,TAN,Neutered,2005-04-14,2018-11-30,NORMAL,OWNER SURRENDER,...,RESCUE,LIVELOVE,33.7997598,-118.1263884,Alive on Intake,False,1,"33.7997598, -118.1263884",27.0,0
2,A707449,*TABITHA,DOG,BLACK,WHITE,Spayed,2022-10-23,2023-09-23,NORMAL,STRAY,...,ADOPTION,NaN,33.7989532,-118.1673340,Alive on Intake,False,1,"33.7989532, -118.167334",18.0,0
3,A712850,*KIWI,DOG,BLONDE,GOLD,Spayed,2022-07-06,2024-02-03,NORMAL,RETURN,...,ADOPTION,WEB,33.7989357,-118.1958891,Alive on Intake,False,1,"33.7989357, -118.1958891",0.0,0
4,A738972,KITTEN 2,CAT,BLACK,NaN,Unknown,2025-03-28,2025-04-04,NORMAL,STRAY,...,RESCUE,LITTLELION,33.7989357,-118.1958891,Alive on Intake,False,1,"33.7989357, -118.1958891",0.0,0


### Structure

In [257]:
# Structure and summary
display(df.dtypes)
display(df.columns)
display(df.describe(include='all'))

Animal ID                    object
Animal Name                  object
Animal Type                  object
Primary Color                object
Secondary Color              object
Sex                          object
DOB                  datetime64[ns]
Intake Date          datetime64[ns]
Intake Condition             object
Intake Type                  object
Intake Subtype               object
Reason for Intake            object
Outcome Date         datetime64[ns]
Crossing                     object
Jurisdiction                 object
Outcome Type                 object
Outcome Subtype              object
latitude                    float64
longitude                   float64
intake_is_dead               object
outcome_is_dead                bool
was_outcome_alive             int64
geopoint                     object
intake_duration             float64
is_current_month              int64
dtype: object

Index(['Animal ID', 'Animal Name', 'Animal Type', 'Primary Color',
       'Secondary Color', 'Sex', 'DOB', 'Intake Date', 'Intake Condition',
       'Intake Type', 'Intake Subtype', 'Reason for Intake', 'Outcome Date',
       'Crossing', 'Jurisdiction', 'Outcome Type', 'Outcome Subtype',
       'latitude', 'longitude', 'intake_is_dead', 'outcome_is_dead',
       'was_outcome_alive', 'geopoint', 'intake_duration', 'is_current_month'],
      dtype='object')

,Animal ID,Animal Name,Animal Type,Primary Color,Secondary Color,Sex,DOB,Intake Date,Intake Condition,Intake Type,...,Outcome Type,Outcome Subtype,latitude,longitude,intake_is_dead,outcome_is_dead,was_outcome_alive,geopoint,intake_duration,is_current_month
count,33707,19956,33707,33707,15964,33707,29433,33707,33707,33707,...,33374,29842,33707.000000000000,33707.000000000000,33707,33707,33707.000000000000,33707,33381.000000000000,33707.000000000000
unique,32557,9996,10,80,44,5,NaN,NaN,16,12,...,18,240,NaN,NaN,1,2,NaN,10154,NaN,NaN
top,A637086,*,CAT,BLACK,WHITE,Male,NaN,NaN,NORMAL,STRAY,...,RESCUE,SPCALA,NaN,NaN,Alive on Intake,False,NaN,"33.8096122, -118.0826161",NaN,NaN
freq,8,104,16083,8548,9380,7739,NaN,NaN,15297,23719,...,7842,4074,NaN,NaN,33707,26766,NaN,570,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,2018-11-03 22:44:42.295383040,2021-02-04 00:22:07.771679488,NaN,NaN,...,NaN,NaN,33.815443890619,-118.149526148085,NaN,NaN,0.794078381345,NaN,18.741949012912,0.012074643249
min,NaN,NaN,NaN,NaN,NaN,NaN,1993-09-15 00:00:00,2017-01-01 00:00:00,NaN,NaN,...,NaN,NaN,19.297814500000,-122.695911500000,NaN,NaN,0.000000000000,NaN,0.000000000000,0.000000000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,2016-09-16 00:00:00,2018-09-29 00:00:00,NaN,NaN,...,NaN,NaN,33.783990200000,-118.190865300000,NaN,NaN,1.000000000000,NaN,0.000000000000,0.000000000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,2019-03-28 00:00:00,2021-01-02 00:00:00,NaN,NaN,...,NaN,NaN,33.806783000000,-118.173175100000,NaN,NaN,1.000000000000,NaN,5.000000000000,0.000000000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,2022-04-06 00:00:00,2023-05-26 00:00:00,NaN,NaN,...,NaN,NaN,33.851209550000,-118.128914550000,NaN,NaN,1.000000000000,NaN,16.000000000000,0.000000000000
max,NaN,NaN,NaN,NaN,NaN,NaN,2025-07-06 00:00:00,2025-07-15 00:00:00,NaN,NaN,...,NaN,NaN,45.521885000000,-73.992359600000,NaN,NaN,1.000000000000,NaN,1410.000000000000,1.000000000000


In [258]:
# Rename columns

def rename(name: str):
    """Formats "name" by replacing spaces with underscores and changing the case to lower

    Args:
        name (str): the name to be formatted

    Returns:
        str: the formatted name
    """    
    name = name.replace(' ', '_')
    name = name.lower()
    if name == 'dob':
        name = 'date_of_birth'
    return name

df = df.rename(columns=rename)
df.columns

Index(['animal_id', 'animal_name', 'animal_type', 'primary_color',
       'secondary_color', 'sex', 'date_of_birth', 'intake_date',
       'intake_condition', 'intake_type', 'intake_subtype',
       'reason_for_intake', 'outcome_date', 'crossing', 'jurisdiction',
       'outcome_type', 'outcome_subtype', 'latitude', 'longitude',
       'intake_is_dead', 'outcome_is_dead', 'was_outcome_alive', 'geopoint',
       'intake_duration', 'is_current_month'],
      dtype='object')

### Variables (columns)

In [259]:
# Organize variables by attributes: animal, intake, outcome, datetime

def check_type_date(name: str):
    """Checks if a column is a datetime or timedelta type by searching the name for keywords

    Args:
        name (str): The string to be checked

    Returns:
        Match|None: A Match object if a match is found
    """      
    return re.search(r'.*date|month|duration.*', name)

animal_vars = [
    'animal_type',
    'primary_color',
    'secondary_color',
    'sex',
]
intake_vars = [x for x in df.columns if 'intake' in x and not check_type_date(x)]
outcome_vars = [x for x in df.columns if 'outcome' in x and not check_type_date(x)]
datetime_vars = [x for x in df.columns if check_type_date(x)]
geography_vars = [
    'latitude',
    'longitude',
    'geopoint',
    'crossing',
    'jurisdiction'
]
print(animal_vars, intake_vars, outcome_vars, datetime_vars, geography_vars, sep='\n')
vars_dict = {
    'animal': animal_vars,
    'intake': intake_vars,
    'outcome': outcome_vars,
    'datetime': datetime_vars,
    'geography': geography_vars
}

['animal_type', 'primary_color', 'secondary_color', 'sex']
['intake_condition', 'intake_type', 'intake_subtype', 'reason_for_intake', 'intake_is_dead']
['outcome_type', 'outcome_subtype', 'outcome_is_dead', 'was_outcome_alive']
['date_of_birth', 'intake_date', 'outcome_date', 'intake_duration', 'is_current_month']
['latitude', 'longitude', 'geopoint', 'crossing', 'jurisdiction']


#### Inspection

In [260]:
# Get counts for each variable and print to a CSV for visual inspection
for vars in vars_dict.values():
    for var in vars:
        df[var].value_counts().to_csv(get_path_obj(data_dir, 'variable counts', f'{var}.csv'))

In [261]:
# Print animal names to CSV for visual inspection
df.loc[~df.animal_name.isna()][['animal_id', 'animal_name']].to_csv(get_path_obj(data_dir, 'raw_animal_names.csv'), index=False)

## Cleaning and preparation

#### Fix/remove data

In [262]:
# Remove superfluous columns
df_clean = df.drop(['was_outcome_alive'], axis=1)
df_clean.columns

Index(['animal_id', 'animal_name', 'animal_type', 'primary_color',
       'secondary_color', 'sex', 'date_of_birth', 'intake_date',
       'intake_condition', 'intake_type', 'intake_subtype',
       'reason_for_intake', 'outcome_date', 'crossing', 'jurisdiction',
       'outcome_type', 'outcome_subtype', 'latitude', 'longitude',
       'intake_is_dead', 'outcome_is_dead', 'geopoint', 'intake_duration',
       'is_current_month'],
      dtype='object')

In [263]:
# Compare latitude and longitude against geopoint

def check_lat_long():
    """Checks the "latitude" and "longitude" columns against the "geopoint" column to see if they match.
    Raises an AssertionError if they do not.
    """    
    df_clean['lat_from_geopoint'] = df.geopoint.str.split(', ').str[0].astype('float64')
    df_clean['long_from_geopoint'] = df.geopoint.str.split(', ').str[1].astype('float64')

    errors = []
    try: 
        assert df_clean.latitude.equals(df_clean.lat_from_geopoint), 'Latitude columns do not match'
    except AssertionError as err:
        errors.append(err)
    try: 
        assert df_clean.longitude.equals(df_clean.long_from_geopoint), 'Longitude columns do not match'
    except AssertionError as err:
        errors.append(err)
    if errors:
        return errors
    else:
        return ['Columns match']

check = check_lat_long()
print(*check, sep='\n')

Longitude columns do not match


In [264]:
# Update latitude, longitude, from geopoint
if check[0] != 'Columns match':
    df_clean.latitude = df_clean.lat_from_geopoint
    df_clean.longitude = df_clean.long_from_geopoint
check = check_lat_long()
print(*check, sep='\n')
if check[0] == 'Columns match':
    df_clean = df_clean.drop(['lat_from_geopoint', 'long_from_geopoint', 'geopoint'], axis=1)

Columns match


In [265]:
# Set specific columns' types to boolean
df_clean.intake_is_dead = df.intake_is_dead.apply(lambda x: False if x in ['Alive on Intake',] else True)
df_clean.is_current_month = df_clean.is_current_month.astype(bool)


In [266]:
# Check intake and outcome dates against intake_duration
df_clean['duration'] = df.outcome_date - df.intake_date
intake_duration_series = pd.to_timedelta(df.intake_duration, unit='day')

assert df_clean.duration.equals(intake_duration_series), '"Intake duration", "Intake date", "Outcome date" are inconsistent'

df_clean.intake_duration = pd.to_timedelta(df.intake_duration, unit='day')
df_clean.drop('duration', axis=1, inplace=True)

In [267]:
# Remove uncaught blank names, names that match IDs, and other names which are not names

#### Add features

In [268]:
# Add an "age at intake" column
df_clean['age_at_intake'] = df.intake_date - df.date_of_birth
df_clean.age_at_intake

0       1096 days
1       4978 days
2        335 days
3        577 days
4          7 days
           ...   
33702   1826 days
33703    149 days
33704   1095 days
33705   1826 days
33706     92 days
Name: age_at_intake, Length: 33707, dtype: timedelta64[ns]

In [269]:
# Add an "age at outcome" column
df_clean['age_at_outcome'] = df.outcome_date - df.date_of_birth
df_clean.age_at_outcome

0       1177 days
1       5005 days
2        353 days
3        577 days
4          7 days
           ...   
33702   1862 days
33703    151 days
33704   1103 days
33705   1832 days
33706    106 days
Name: age_at_outcome, Length: 33707, dtype: timedelta64[ns]

In [270]:
# Add a "has_name" column
df_clean['has_name'] = ~df.animal_name.isna()

In [271]:
# Add a "named_by_shelter" column
df_clean['named_by_shelter'] = df.animal_name.str.contains(r'\*').astype(bool)
df_clean

,animal_id,animal_name,animal_type,primary_color,secondary_color,sex,date_of_birth,intake_date,intake_condition,intake_type,...,latitude,longitude,intake_is_dead,outcome_is_dead,intake_duration,is_current_month,age_at_intake,age_at_outcome,has_name,named_by_shelter
0,A594350,*HEAVY CREAM,CAT,BLACK,NaN,Neutered,2014-07-28,2017-07-28,NORMAL,STRAY,...,33.7997598,-118.1263884,False,False,81 days,False,1096 days,1177 days,True,True
1,A347815,DUKE,DOG,BLACK,TAN,Neutered,2005-04-14,2018-11-30,NORMAL,OWNER SURRENDER,...,33.7997598,-118.1263884,False,False,27 days,False,4978 days,5005 days,True,False
2,A707449,*TABITHA,DOG,BLACK,WHITE,Spayed,2022-10-23,2023-09-23,NORMAL,STRAY,...,33.7989532,-118.1673340,False,False,18 days,False,335 days,353 days,True,True
3,A712850,*KIWI,DOG,BLONDE,GOLD,Spayed,2022-07-06,2024-02-03,NORMAL,RETURN,...,33.7989357,-118.1958891,False,False,0 days,False,577 days,577 days,True,True
4,A738972,KITTEN 2,CAT,BLACK,NaN,Unknown,2025-03-28,2025-04-04,NORMAL,STRAY,...,33.7989357,-118.1958891,False,False,0 days,False,7 days,7 days,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33702,A591444,*MATILDA,DOG,BLACK,NaN,Female,2012-06-06,2017-06-06,ILL MODERATETE,STRAY,...,33.8043164,-118.1894505,False,False,36 days,False,1826 days,1862 days,True,True
33703,A702835,*NESTLE,DOG,BLACK,WHITE,Neutered,2023-02-14,2023-07-13,NORMAL,OWNER SURRENDER,...,33.8042880,-118.2151140,False,False,2 days,False,149 days,151 days,True,True
33704,A699371,*GIZMO,DOG,WHITE,TAN,Male,2020-05-25,2023-05-25,NORMAL,STRAY,...,33.8414460,-118.1162032,False,False,8 days,False,1095 days,1103 days,True,True
33705,A630046,NaN,DOG,TAN,NaN,Female,2014-07-17,2019-07-17,NORMAL,STRAY,...,33.8187528,-118.1163116,False,False,6 days,False,1826 days,1832 days,False,True


In [272]:
# Check data types
df_clean.dtypes

animal_id                     object
animal_name                   object
animal_type                   object
primary_color                 object
secondary_color               object
sex                           object
date_of_birth         datetime64[ns]
intake_date           datetime64[ns]
intake_condition              object
intake_type                   object
intake_subtype                object
reason_for_intake             object
outcome_date          datetime64[ns]
crossing                      object
jurisdiction                  object
outcome_type                  object
outcome_subtype               object
latitude                     float64
longitude                    float64
intake_is_dead                  bool
outcome_is_dead                 bool
intake_duration      timedelta64[ns]
is_current_month                bool
age_at_intake        timedelta64[ns]
age_at_outcome       timedelta64[ns]
has_name                        bool
named_by_shelter                bool
d

## Exploratory Data Analysis (EDA)

## Deeper analysis and modeling

# Analysis

## Insights and recommendations

### Insights

### Recommedations

## Summary

*This report can also be found [here](../products/report.md).*

## Appendix